<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/main/interact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From interact.py file

In [3]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.6MB/s 
     |████████████████████████████████| 3.2MB 47.2MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=f5a15472b1ed78a38164c36dee66aa9bf642b82c1f330d0aff51ed134c4381f7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, top_k_top_p_filtering
import re
from torch.nn import functional as F

In [12]:
modelPath = "drive/MyDrive/Honours2021/TNGv3"

model = GPT2LMHeadModel.from_pretrained(modelPath)
tokenizer = GPT2Tokenizer.from_pretrained(modelPath)

input_str = "PICARD: I am Captain Picard, commanding the Enterprise. PICARD: You will agree, Data, that Starfleet's orders are difficult? DATA: Difficult? Simply solve the mystery of Farpoint Station."

personaID = "<bos> TROI: I am Troi."
persona2 = "RIKER: I am Commander William T Riker."


def generate_output(history):
    sequence = history + " TROI:"

    for i in range(150):

        input_ids = tokenizer.encode(sequence, return_tensors='pt')

        # get logits of last hidden state
        next_token_logits = model(input_ids).logits[:, -1, :]

        # filter
        filtered_next_token_logits = top_k_top_p_filtering(next_token_logits, top_k=50, top_p=0.95)

        # sample
        probs = F.softmax(filtered_next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        n_token = next_token.tolist()[0][0]
        bos_token_id = tokenizer.bos_token_id
        if n_token == tokenizer.pad_token_id or n_token == bos_token_id:
            continue

        generated = torch.cat([input_ids, next_token], dim=-1)

        resulting_string = tokenizer.decode(generated.tolist()[0])

        #last_token = tokenizer.decode(next_token[0])

        sequence = resulting_string
        if sequence.rindex(" ") > sequence.rindex("."):
          last_word = sequence[sequence.rindex(" ") + 1:]
        else:
          last_word = sequence[sequence.rindex(".") + 1:]
  
        if re.match("([A-Z]+ ?[A-Z]+ ?:)|[A-Z]:", last_word) or last_word in tokenizer.eos_token:
            print("matched")
            resulting_string = resulting_string[: sequence.rindex(last_word)]
            break;

    #just generated outcome
    resulting_string = resulting_string[resulting_string.rindex("TROI:"):]
    return resulting_string


#give me for examples with same string
#for i in range(4):
#    generate_output(input_str)

#start chat.
history = [personaID, persona2]

for i in range(5):

    userinput = input("RIKER: ")

    history.append("RIKER: " + userinput)
    context = " ".join(history)
    output = generate_output(context)
    history.append(output)
    print(output)
    #labels we can do labels of pad

RIKER: Good day Dianna.
matched
TROI: I'm Deanna Troi, Ship's Counsellor.
RIKER: What do you think about the enterprice.
matched
TROI: I'm curious.
RIKER: What is your name?
matched
TROI: I'm William Riker.
RIKER: no. I am William Riker.
matched
TROI: Is there anything you'd like me to say? 
RIKER: Are you Dianna Troi?
matched
TROI: How long ago was it? 
